In [ ]:
import re
import csadata.twitter as twitter
from datetime import datetime, timezone

The `TwitterAPI` class represents a handle to the Twitter Tweet Search API. Since each API request has to be authorized,
we have to transmit an API token via the `bearer_token` parameter.

In [ ]:
bearer_token = None
api = twitter.TwitterAPI(bearer_token=bearer_token)

The `TweetCSVWriter` class is responsible for writing the contents and metrics of the requested Tweets to indexed CSV
files. Each file holds the amount of Tweets that have been posted within a certain time interval. The duration of this
time interval in seconds is expressed by the `file_interval` parameter. A file indexing structure can be built with time
intervals that are greater than the one described by the `file_interval` parameter. Each level of this indexing
structure is also described by a time interval. The `dir_intervals` parameter holds these time interval durations (also
in seconds) in descending order.

In this example, we structure the downloaded Tweet data in CSV files with a corresponding time interval of 1 hour and
in directories with a corresponding time interval of 1 day.

In [ ]:
dir_interval = 60 * 60 * 24
file_interval = 60 * 60

# 06/01/2021 00:00:00
start = int(datetime(2021, 6, 1, 0, 0, 0, tzinfo=timezone.utc).timestamp())

# 06/01/2021 23:59:59
end = int(datetime(2021, 6, 1, 23, 59, 59, tzinfo=timezone.utc).timestamp())


Each call to the Twitter Tweet Search API is characterized by a query. Please refer to the Twitter API documentation in
order to get information on how the query string is built.

In [ ]:
query = "-is:retweet (dogecoin OR doge) (lang:en OR lang:und)"

with twitter.TweetCSVWriter("data", dir_intervals=[dir_interval], file_interval=file_interval) as csv_writer:
    for tweet in api.search(endpoint=twitter.RECENT_SEARCH_ENDPOINT, query=query, max_requests=1, start_time=start,
                            end_time=end):
        # Remove links
        tweet.text = re.compile(r"https?://\S+").sub("", tweet.text)
        csv_writer.write(tweet)

In [ ]:
with twitter.TweetCSVReader(base_path="data", dir_intervals=[dir_interval], file_interval=file_interval, start=start,
                            end=end) as csv_reader:
    for tweet in csv_reader:
        print(tweet)